#Part 1: CNN

# A) Importing Modules For CNN Model

In [ ]:
#SECTION 1.1.1 Importing Modules For CNN Model
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# B) Loading the Data

In [ ]:
#SECTION 1.1.2 Loading the Data

IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 32

current_directory = os.getcwd()

train_dataset_path = os.path.join(current_directory, 'mosquitos_train')
validation_dataset_path = os.path.join(current_directory, 'mosquitos_val')
test_dataset_path = os.path.join(current_directory, 'mosquitos_test')



In [ ]:
#Section 1.2.1 Reading the Images and Sorting Based on Class
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   shuffle=True)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dataset_path,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                             class_mode='categorical',
                                                             shuffle=True)

In [ ]:
#Section 1.2.2 Labeling Each Image by Label
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

In [ ]:
#Section 1.2.3 Displaying Sample Images via Label
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(15, 12))
idx = 0

for i in range(2):
    for j in range(5):
        label = labels[np.argmax(train_generator[0][1][idx])]
        ax[i, j].set_title(f"{label}")
        ax[i, j].imshow(train_generator[0][0][idx][:, :, :])
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Sample Training Images", fontsize=21)
plt.show()

# C) Creating a Convolution Neural Network

In [ ]:
  #Section 1.3.1 Creating a CNN Model
#keras.utils.set_random_seed(812)
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='same', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='same', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='same', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ])

    return model

In [ ]:
cnn_model = create_model()

In [ ]:
print(cnn_model.summary())

# D) Training the Model

In [ ]:
#Section 1.3.2 Reducing Learning Rate And Applying a Stochastic Gradient Descent
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)
optimizer = Adam(learning_rate=0.001)
cnn_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
#Section 1.3.3 Performing Supervised Learning
history = cnn_model.fit(train_generator, epochs=40, validation_data=validation_generator,
                       verbose=2,
                       callbacks=[reduce_lr])

In [ ]:
#Section 1.4.1 Viewing The Model History and Accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

learning_rate = history.history['lr']

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12, 10))

ax[0].set_title('Training Accuracy vs. Epochs')
ax[0].plot(train_accuracy, 'o-', label='Train Accuracy')
ax[0].plot(val_accuracy, 'o-', label='Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(loc='best')

ax[1].set_title('Training/Validation Loss vs. Epochs')
ax[1].plot(train_loss, 'o-', label='Train Loss')
ax[1].plot(val_loss, 'o-', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(loc='best')

ax[2].set_title('Learning Rate vs. Epochs')
ax[2].plot(learning_rate, 'o-', label='Learning Rate')
ax[2].set_xlabel('Epochs')
ax[2].set_ylabel('learing rate')
ax[2].legend(loc='best')

plt.tight_layout()
plt.show()

# E) Testing the Model

In [ ]:
#Section 1.5.1 Running the Model With Test Data
test_dataset = os.path.join(current_directory, 'mosquitos_test')
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(test_dataset,
                                                 shuffle=False,
                                                 batch_size=BATCH_SIZE,
                                                 target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                 class_mode='categorical')

test_datasetsample = os.path.join(current_directory, 'mosquitos_test_sample')
test_datagensample = ImageDataGenerator(rescale=1.0/255)

test_generatorsample = test_datagensample.flow_from_directory(test_datasetsample,
                                                 shuffle=False,
                                                 batch_size=BATCH_SIZE,
                                                 target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                 class_mode='categorical')

In [ ]:
#Section 1.5.2 Displaying Predictions with Random Test Photos
predictions = cnn_model.predict(test_generator)
cnn_loss,cnn_accuracy = cnn_model.evaluate(test_generator)
class_indices = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(12, 10))
idx = 0
for i in range(2):
  for j in range(5):
    predicted_label = labels[np.argmax(predictions[idx*32])]
    actual_class_index = class_indices[idx*32]
    actual_class_label = class_labels[actual_class_index]
    ax[i,j].set_title(f"Predicted: {predicted_label}\nActual: {actual_class_label}")
    ax[i,j].imshow(test_generator[idx][0][idx])
    ax[i,j].axis("off")
    idx += 1

plt.tight_layout()
plt.suptitle("Test Dataset Predictions", fontsize=20)
plt.show()
print("test accuracy:", cnn_accuracy)



In [ ]:
#Section 1.5.3 Displaying Predictions with Set Test Photos
predictions_sample = cnn_model.predict(test_generatorsample)
class_indices_sample = test_generatorsample.classes
class_labels_sample = list(test_generatorsample.class_indices.keys())
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(12, 10))
idx = 0
for j in range(6):
  score=np.argmax(predictions_sample[idx])
  predicted_label = labels[score]
  actual_class_index = class_indices_sample[idx]
  actual_class_label = class_labels_sample[actual_class_index]
  ax[j].set_title(f"{predicted_label}\nActual: {actual_class_label}")
  ax[j].imshow(test_generatorsample[0][0][idx])
  ax[j].axis("off")
  idx += 1

plt.tight_layout()
plt.suptitle("Test Dataset Predictions", fontsize=20)
plt.show()

# Part 2: Decision Tree

# A) Importing Modules and Data
 

In [ ]:
#Section 2.1.1 Importing Modules for Decision Tree Model
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
import cv2

rng_seed=1337

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    seed=1337,
    image_size=(150,150),
    batch_size=128,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dataset_path,
    seed=1337,
    image_size=(150,150),
    batch_size=128
)

# B) Converting Images to Numpy Arrays

In [ ]:
#Section 2.1.2 Initializing the Data Set
# Initialize lists to store images and labels
images_list = []
labels_list = []

# Iterate through the prefetched dataset and collect images and labels
for images, labels in train_ds:
    images_list.append(images.numpy())
    labels_list.append(labels.numpy())

# Concatenate images and labels into NumPy arrays
images_array = np.concatenate(images_list)
labels_array = np.concatenate(labels_list)
y_train=labels_array


In [ ]:
#Section 2.1.3
# Initialize lists to store images and labels
images_list2 = []
labels_list2 = []

# Iterate through the prefetched dataset and collect images and labels
for images, labels in test_ds:
    images_list2.append(images.numpy())
    labels_list2.append(labels.numpy())

# Concatenate images and labels into NumPy arrays
images_array2 = np.concatenate(images_list2)
labels_array2 = np.concatenate(labels_list2)
y_test=labels_array2
x_test=images_array2

In [ ]:
#Section 2.1.4 Normalizing and Shaping the Data
# Normalization
x_train = images_array/255.0

#sklearn expects i/p to be 2d array-model.fit(x_train,y_train)=>reshape to 2d array
nsamples, nx, ny, nrgb = x_train.shape
x_train2 = x_train.reshape((nsamples,nx*ny*nrgb))


# C) Run a Single Tree

In [ ]:
#Section 2.2.1 Running a single decision tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
dtc_model=DecisionTreeClassifier()
dtc_model.fit(x_train2,y_train)

plt.figure(figsize=(12, 8))
plot_tree(dtc_model, filled=True, rounded=True)
plt.show()

In [ ]:
#Section 2.2.2 Preparing the Test Data
nsamples, nx, ny, nrgb = x_test.shape
x_test2 = x_test.reshape((nsamples,nx*ny*nrgb))

In [ ]:
#Section 2.3.1
y_pred_dtc=dtc_model.predict(x_test2)
accuracy_score(y_pred_dtc,y_test)
print(classification_report(y_pred_dtc,y_test))

# D) Multiple Decision Trees (Do Not Use!)

In [ ]:
#Section 2.4 Running multiple decision trees
import scipy

N_Trees = 150
dtc_acc = np.empty(N_Trees)
yhat = np.empty((N_Trees,x_test2.shape[0]))

trees = list()
for i in range(N_Trees):
    ind = np.random.choice(range(x_train2.shape[0]),x_train2.shape[0])
    x_bs = x_train2[ind,:]
    y_bs = y_train[ind]

    dtc_model = DecisionTreeClassifier().fit(x_bs,y_bs)
    trees.append(dtc_model)

    yhat[i,:] = dtc_model.predict(x_test2)
    dtc_acc[i] = accuracy_score(y_test,yhat[i,:])

#Results
y_ensemble = scipy.stats.mode(yhat, keepdims=True).mode[0]
print(classification_report(y_ensemble,y_test))
print(accuracy_score(y_ensemble,y_test))

